# MNIST a través de redes neuronales

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('./MNIST_data/', one_hot = True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


# Helper functions

## Inicializar pesos

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 1.0)
    return tf.Variable(init_random_dist)

## Inicializar bias 

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

## CONV2D 

In [6]:
def conv2d(x, W):
    # x : [batch, H, W, Channels]
    # W (kernel) : [filter H, filter W, Channels IN, Channels OUT]
    
    #Los strides en las esquinas son 1 porque no saltea nada en esas dimensiones, los dos elementos centrales pueden
    #ser diferentes
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

## Pooling 

In [7]:
def max_pool_2by2(x):
    # x es [batch, h, w, c]
    
    #El pooling es en H y W (lo más común) pero se podría hacer en las demás dimensiones
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

## Convolutional Layer

In [8]:
def convolutional_layer(input_x, shape):
    # shape = [#tam1 #tam2 #entrada #salida]
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

## Normal (FULLY CONNECTED)

In [9]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

## Placeholders

In [10]:
x = tf.placeholder(tf.float32, shape = [None, 784]) # la imagen es de 28x28 = 784

In [11]:
y_true = tf.placeholder(tf.float32, shape=[None, 10]) #Vamos a discriminar entre 10 números diferentes

## Layers

In [12]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # [batch, h, w, c] => el -1 aplana

In [13]:
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32]) #5x5, 1 canal y 32 features => capas de salida
convo_1_pooling = max_pool_2by2(convo_1)

In [14]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])#5x5, 32 canales, 64 features
convo_2_pooling = max_pool_2by2(convo_2)

In [15]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

## Dropout

In [16]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [17]:
y_pred = normal_full_layer(full_one_dropout, 10)

## Loss Function

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

## Optimizer

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [20]:
init = tf.global_variables_initializer()

In [21]:
saver = tf.train.Saver(max_to_keep=None) #Me aseguro de que guarde todos los modelos

In [22]:
#def save(self, epoch): #Lo saqué de por ahí
#    model_name = "MNIST-model"
#    checkpoint_dir = os.path.join(model_name)

#    if not os.path.exists(checkpoint_dir):
#        os.makedirs(checkpoint_dir)
#    self.saver.save(self.sess, checkpoint_dir + '/model', global_step=epoch)
#    self.saver.save(self.sess, checkpoint_dir + '/model')
#    print("path for saved %s" % checkpoint_dir)

## Lo siguiente está incompleto, si por bien cada vez que lo corro graba la red entremada, sobreescribe la anterior sin usar checkpoints

### Corregir en cuánto pueda

In [23]:
steps = 2000

noentrenarporquetardamuchoerror

#import os

with tf.Session() as sess:
    
#    model_name = "MNIST-model"
#    checkpoint_dir = os.path.join(model_name)
    
#    if not os.path.exists(checkpoint_dir):
#        os.makedirs(checkpoint_dir)
    
    sess.run(init)
    
#     saver.restore(sess, './modelTest/my_second_model.ckpt')
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict = {x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            
            print("ON STEP: {}".format(i))
            print("Accuracy: ")
            
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))

            print(sess.run(acc, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            
    saver.save(sess,'./modelTest/my_second_model.ckpt')
    #self.save(i)

NameError: name 'noentrenarporquetardamuchoerror' is not defined

In [22]:
# Pruebo restaurar la sesión y correr las pruebas

with tf.Session() as sess:
    
#    model_name = "MNIST-model"
#    checkpoint_dir = os.path.join(model_name)
    
#    if not os.path.exists(checkpoint_dir):
#        os.makedirs(checkpoint_dir)
    
    sess.run(init)
    
    saver.restore(sess, './modelTest/my_second_model.ckpt')

    matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
    acc = tf.reduce_mean(tf.cast(matches, tf.float32))

    print('Exactitud del ejemplo # 21')
    print(sess.run(acc, feed_dict = {x:[mnist.test.images[20]], y_true:[mnist.test.labels[20]], hold_prob:1.0}))
    
    print('Exactitud de los ejemplos # 101 al 201')
    print(sess.run(acc, feed_dict = {x:mnist.test.images[100:200], y_true:mnist.test.labels[100:200], hold_prob:1.0}))

#     print('Exactitud promedio de todos los ejemplos de prueba')
#     print(sess.run(acc, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
    
#     confusion = tf.confusion_matrix(labels=tf.argmax(y_pred,1), predictions=tf.argmax(y_true,1), num_classes=10, dtype=tf.float32)
#     c_matrix = sess.run(confusion, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels})
    
#     confusion = tf.confusion_matrix(labels=tf.argmax(y_pred,1), predictions=tf.argmax(y_true,1), num_classes=10, dtype=tf.float32)
    
#     c_matrix = sess.run(confusion, feed_dict = {x:mnist.test.images, y_true:mnist.test.labels})


INFO:tensorflow:Restoring parameters from ./modelTest/my_second_model.ckpt
Exactitud del ejemplo # 21
1.0
Exactitud de los ejemplos # 101 al 201
0.97


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    iol

In [25]:
y_pred.dtype

tf.float32

In [26]:
y_true.dtype

tf.float32

In [ ]:
c_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
idx = 10
plt.imshow(mnist.test.images[idx].reshape(28, 28))

In [ ]:
j=0
for i in np.random.randint(0,len(mnist.test.images)-1,100):
    plt.subplot(10,10,j+1)
    plt.imshow(mnist.test.images[i].reshape(28, 28))
    j = j+1

In [27]:
mnist.test.images[0].shape

(784,)